# Chapter 1: The PatientOne Story

**Companion notebook for Chapter 1 of _Building AI-Orchestrated Precision Oncology Systems_**

---

## Prerequisites

**IMPORTANT**: This notebook requires you to deploy your own MCP servers to GCP Cloud Run.

Setup instructions: See **Appendix: MCP Server Setup Guide** or [`companion-notebooks/README.md`](./README.md)

**You will need**:
- GCP project with Cloud Run enabled
- All 12 MCP servers deployed (see Appendix)
- Anthropic API key or Google AI API key
- PatientOne dataset (included in repository)

In [ ]:
# Configuration
import os
from anthropic import Anthropic

# Set your API key
os.environ["ANTHROPIC_API_KEY"] = "your_key_here"  # Replace

# Your deployed MCP server URLs (replace YOUR_PROJECT)
MCP_SERVERS = {
    "fgbio": "https://mcp-fgbio-YOUR_PROJECT.run.app/sse",
    "multiomics": "https://mcp-multiomics-YOUR_PROJECT.run.app/sse",
    "spatialtools": "https://mcp-spatialtools-YOUR_PROJECT.run.app/sse"
}

client = Anthropic()
print("✅ Configuration loaded")

## Exercise 1: Calculate Time & Cost Savings

In [ ]:
# Time reduction
traditional_hours = 40
ai_minutes = 35
speedup = (traditional_hours * 60) / ai_minutes

print(f"Traditional: {traditional_hours} hours")
print(f"AI-orchestrated: {ai_minutes} minutes")
print(f"Speedup: {speedup:.1f}x faster")

# Cost reduction
trad_cost = 3200
ai_cost = 1.20
savings = (trad_cost - ai_cost) / trad_cost * 100

print(f"\nTraditional cost: ${trad_cost}")
print(f"AI cost: ${ai_cost}")
print(f"Savings: {savings:.2f}%")

## Exercise 2: Explore PatientOne Dataset

In [ ]:
from pathlib import Path

data_dir = Path("../../../data/patient-data/PAT001-OVC-2025")

if data_dir.exists():
    print("PatientOne Dataset:")
    for subdir in sorted(data_dir.iterdir()):
        if subdir.is_dir():
            files = list(subdir.iterdir())
            print(f"\n{subdir.name}/: {len(files)} files")
else:
    print("⚠️  Data not found. Clone full repository.")